In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
# base data inform and network data
input_size = 4
hidden_size = 24
num_classes = 3
num_epochs = 10
batch_size = 4
learning_rate = 0.01

In [26]:
from sklearn import datasets

class IrisDataset(Dataset):

    # data loading
    def __init__(self):
        iris = datasets.load_iris()
        feature = pd.DataFrame(iris.data, columns=iris.feature_names)
        target = pd.DataFrame(iris.target, columns=['target'])
        iris_data = pd.concat([target, feature], axis=1)
        # Data type change and flatten targets
        self.x = torch.from_numpy(np.array(iris_data)[:, 1:].astype(np.float32))
        self.y = torch.from_numpy(np.array(iris_data)[:, [0]].astype(np.longlong).flatten())
        self.n_samples = self.x.shape[0]

    # working for indexing
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]

    # return the length of our dataset
    def __len__(self):

        return self.n_samples


dataset = IrisDataset()

# create data spliter
def dataSplit(dataset, val_split=0.25, shuffle=False, random_seed=0):

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    train_indices, val_indices = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    return train_sampler, valid_sampler

# base split parameters
val_split = 0.25
shuffle_dataset = True
random_seed= 42

train_sampler, valid_sampler = \
    dataSplit(dataset=dataset, val_split=val_split, shuffle=shuffle_dataset, random_seed=random_seed)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [27]:
#建立 FeedForward

# 1) model build
class FeedForwardNeuralNet(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedForwardNeuralNet, self).__init__()
        # define first layer
        self.l1 = nn.Linear(input_size, hidden_size)
        # activation function
        self.relu = nn.ReLU()
        # define second layer
        self.l2 = nn.Linear(hidden_size, hidden_size)
        # define third layer
        self.l3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)

        return out


model = FeedForwardNeuralNet(input_size, hidden_size, num_classes)

In [28]:
#選擇適合的 loss function 和 optimizer
# 2) loss and optimizer
# Cross Entropy
criterion = nn.CrossEntropyLoss()
# adam algorithm
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
# 3) Training loop

FILE = 'model_all.pt'
min_loss = 1

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (datas, labels) in enumerate(train_loader):
        
        datas = datas.to(device)
        labels = labels.to(device)

        # init optimizer
        optimizer.zero_grad()
        
        # forward -> backward -> update
        outputs = model(datas)
        loss = criterion(outputs, labels)
        
        loss.backward()

        optimizer.step()

        if loss.item() < min_loss:
              min_loss = loss.item()
              torch.save(model.state_dict(), FILE)

        if (i + 1) % 29 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1/10, step 29/29, loss = 0.0154
epoch 2/10, step 29/29, loss = 0.0000
epoch 3/10, step 29/29, loss = 0.0000
epoch 4/10, step 29/29, loss = 0.0024
epoch 5/10, step 29/29, loss = 0.0018
epoch 6/10, step 29/29, loss = 0.0204
epoch 7/10, step 29/29, loss = 0.0013
epoch 8/10, step 29/29, loss = 0.0217
epoch 9/10, step 29/29, loss = 0.0097
epoch 10/10, step 29/29, loss = 0.0147


In [35]:
# 4) Testing

model = FeedForwardNeuralNet(input_size, hidden_size, num_classes)
model.load_state_dict(torch.load(FILE))
model.eval()

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for datas, labels in val_loader:

        datas = datas.to(device)
        labels = labels.to(device)

        outputs = model(datas.float())
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc}')

accuracy = 97.29729729729729
